    Intents.json – The data file which has predefined patterns and responses.
    train_chatbot.py – In this Python file, we wrote a script to build the model and train our chatbot.
    Words.pkl – This is a pickle file in which we store the words Python object that contains a list of our vocabulary.
    Classes.pkl – The classes pickle file contains the list of categories.
    Chatbot_model.h5 – This is the trained model that contains information about the model and has weights of the neurons.
    Chatgui.py – This is the Python script in which we implemented GUI for our chatbot. Users can easily interact with the bot.

Here are the 5 steps to create a chatbot in Python from scratch:

    Import and load the data file
    Preprocess data
    Create training and testing data
    Build the model
    Predict the response

1. Import and load the data file

First, make a file name as train_chatbot.py. We import the necessary packages for our chatbot and initialize the variables we will use in our Python project.

The data file is in JSON format so we used the json package to parse the JSON file into Python.

In [6]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import tensorflow as ts
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
import nltk
nltk.download('punkt')
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\91974\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\91974\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

## This is how our intents.json file looks like.

In [8]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day'],
   'responses': ['Hello, thanks for asking',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Nice chatting to you, bye',
    'Till next time'],
   'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'],
   'context': ['']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'context': ['']},
  {'tag': 'noanswer',
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I understand'],
   'context': ['']},
  {'tag': 'options',
   'patterns': ['How you could help me?',
    'What you can do

## 2. Preprocess data

### When working with text data, we need to perform various preprocessing on the data before we make a machine learning or a deep learning model. Based on the requirements we need to apply various operations to preprocess the data.

### Tokenizing is the most basic and first thing you can do on text data. Tokenizing is the process of breaking the whole text into small parts like words.

### Here we iterate through the patterns and tokenize the sentence using nltk.word_tokenize() function and append each word in the words list. We also create a list of classes for our tags.

In [9]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

### Now we will lemmatize each word and remove duplicate words from the list. Lemmatizing is the process of converting a word into its lemma form and then creating a pickle file to store the Python objects which we will use while predicting.

In [10]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [11]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

### 3. Create training and testing data

Now, we will create the training data in which we will provide the input and the output. Our input will be the pattern and output will be the class our input pattern belongs to. But the computer doesn’t understand text so we will convert text into numbers.

In [12]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


<ipython-input-12-d4d4ba6c3a48>:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [15]:
len(train_x[0])

88

### 4. Build the model

We have our training data ready, now we will build a deep neural network that has 3 layers. We use the Keras sequential API for this. After training the model for 200 epochs, we achieved 100% accuracy on our model. Let us save the model as ‘chatbot_model.h5’.

In [16]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [17]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

C:\Users\91974\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/200
10/10 [==============================] - 12s 2ms/step - loss: 2.1639 - accuracy: 0.1159
Epoch 2/200
10/10 [==============================] - 0s 2ms/step - loss: 2.1146 - accuracy: 0.2221
Epoch 3/200
10/10 [==============================] - 0s 2ms/step - loss: 2.0510 - accuracy: 0.2908
Epoch 4/200
10/10 [==============================] - 0s 2ms/step - loss: 2.0389 - accuracy: 0.3189
Epoch 5/200
10/10 [==============================] - 0s 2ms/step - loss: 1.8266 - accuracy: 0.3730
Epoch 6/200
10/10 [==============================] - 0s 2ms/step - loss: 1.6657 - accuracy: 0.5895
Epoch 7/200
10/10 [==============================] - 0s 2ms/step - loss: 1.5936 - accuracy: 0.4734
Epoch 8/200
10/10 [==============================] - 0s 2ms/step - loss: 1.4438 - accuracy: 0.6168
Epoch 9/200
10/10 [==============================] - 0s 2ms/step - loss: 1.3101 - accuracy: 0.5962
Epoch 10/200
10/10 [==============================] - 0s 3ms/step - loss: 1.2524 - accuracy: 0.4882
Epoch 11

10/10 [==============================] - 0s 2ms/step - loss: 0.0273 - accuracy: 1.0000
Epoch 84/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0198 - accuracy: 1.0000
Epoch 85/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0278 - accuracy: 1.0000
Epoch 86/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0480 - accuracy: 0.9862
Epoch 87/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0551 - accuracy: 0.9892
Epoch 88/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0945 - accuracy: 0.9862
Epoch 89/200
10/10 [==============================] - 0s 2ms/step - loss: 0.3492 - accuracy: 0.9076
Epoch 90/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0664 - accuracy: 1.0000
Epoch 91/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0811 - accuracy: 0.9720
Epoch 92/200
10/10 [==============================] - 0s 1ms/step - loss: 0.1113 - accuracy: 0.9780
Epoch 93/200


10/10 [==============================] - 0s 2ms/step - loss: 0.0181 - accuracy: 1.0000
Epoch 165/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0072 - accuracy: 1.0000
Epoch 166/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0093 - accuracy: 1.0000
Epoch 167/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0396 - accuracy: 1.0000
Epoch 168/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0128 - accuracy: 1.0000
Epoch 169/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0118 - accuracy: 1.0000
Epoch 170/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0179 - accuracy: 0.9941
Epoch 171/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0139 - accuracy: 1.0000
Epoch 172/200
10/10 [==============================] - 0s 1ms/step - loss: 0.0619 - accuracy: 0.9826
Epoch 173/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0382 - accuracy: 0.9961
Epoc

### 5. Predict the response (Graphical User Interface)

To predict the sentences and get a response from the user to let us create a new file ‘chatapp.py’.

We will load the trained model and then use a graphical user interface that will predict the response from the bot. The model will only tell us the class it belongs to, so we will implement some functions which will identify the class and then retrieve us a random response from the list of responses.

Again we import the necessary packages and load the ‘words.pkl’ and ‘classes.pkl’ pickle files which we have created when we trained our model: